# Cats vs Dogs Classification using CNN

Dataset: Cats vs Dogs

Model: CNN with 3 convolutional layers

Loss Function: Binary Crossentropy

Optimizer: Adam

Output: Interactive Gradio app for real-time predictions

## Step 1: Extract Dataset

In [ ]:
import os
import zipfile
zip_path = '/content/archive.zip'   # change this to your actual filename


In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('data/')   # this will create a folder named "data"


In [ ]:
os.listdir('data')


## Step 2: Load and Prepare Data

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [ ]:
# Generaters
# making batches for large datasets
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/data/train' ,
    labels = 'inferred',
    label_mode = 'int',
    batch_size= 32,
    image_size=(256,256)
)

In [ ]:
validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/data/test' ,
    labels = 'inferred',
    label_mode = 'int',
    batch_size= 32,
    image_size=(256,256)
)

## Step 3: Normalize Pixel Values

In [ ]:
#Normalize Pixel value b/w 0 to 1
def process(image,label):
  image = tf.cast(image/255. ,tf.float32)
  return image,label

  train_ds = train_ds.map(process)
  validation_ds = validation_ds.map(process)

## Step 4: Exploratory Data Analysis (EDA)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Check basic info
print("Number of training batches:", tf.data.experimental.cardinality(train_ds).numpy())
print("Number of validation batches:", tf.data.experimental.cardinality(validation_ds).numpy())

# Get class names
class_names = train_ds.class_names
print("Class names:", class_names)

# Count number of images per class
count_dict = {class_name: 0 for class_name in class_names}
for images, labels in train_ds.unbatch():
    count_dict[class_names[int(labels.numpy())]] += 1
print("Image count per class:", count_dict)

# Plot class distribution
plt.bar(count_dict.keys(), count_dict.values())
plt.title("Class Distribution in Training Data")
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.show()

# Show some sample images
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

# Check image shape and data type
for image_batch, label_batch in train_ds.take(1):
    print("Image batch shape:", image_batch.shape)
    print("Label batch shape:", label_batch.shape)
    print("Pixel value range:", tf.reduce_min(image_batch).numpy(), "to", tf.reduce_max(image_batch).numpy())


In [ ]:
print(train_ds.class_names)
for images, labels in train_ds.take(1):
    print(labels[:10].numpy())  # show first 10 labels
    break
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")
plt.show()


## Step 5: Build CNN Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

# Create CNN model
# there would be 3 conv layers
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

## Step 6: Compile and Train Model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

## Step 7: Deploy with Gradio

In [ ]:
!pip install gradio
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing import image

# If model is already in memory from training, no need to reload
# (Otherwise, you can load it like this:)
# from tensorflow.keras.models import load_model
# model = load_model("model.h5")

# Class names as per your dataset
class_names = ['cats', 'dogs']  # adjust if directory names differ

# Prediction function for Gradio
def predict_image(img):
    # Resize and preprocess
    img = img.resize((256, 256))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # Predict
    pred = model.predict(img)[0][0]

    # Convert to label and confidence
    label = class_names[int(pred > 0.5)]
    confidence = pred if label == 'dogs' else 1 - pred

    return {label: float(confidence)}

# Create Gradio interface
demo = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Cat or Dog Image "),
    outputs=gr.Label(num_top_classes=2),
    title=" Cats vs Dogs Classifier",
    description="Upload an image to see if it's a cat or a dog.",
)

demo.launch()


## Step 8: Visualize Training Performance

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show